#### Global & Local Modules

In [1]:
import os
from itertools import permutations 
import sys
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from rgcn import GaussianGraphConvolution, kl_reg

Using TensorFlow backend.


#### Vendor Modules

In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS
# import vendor modules
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds

# Data

In [3]:
def attack_edges(attack_ratio: float) -> 'callable':
    """
    Create an edge attack function with given attack ratio.

    Args:
        attack_ratio: the ratio of noise to clean edges

    Returns:
        a callable function for attacking a graph

    """
    def _attack_edges(edges: np.ndarray) -> np.ndarray:
        f"""
        Attack the edges of the given list of edges.

        Args:
            edges: the ndarray of tuples representing edges to attack

        Returns:
            a new ndarray of edges with random edges added

        Note:
            the ratio of noise to clean edges is {attack_ratio}

        """
        # create the set of all possible edges
        all_edges = set(permutations(list(range(edges.max())), 2))
        # convert the ndarray of edges to a set of tuples
        edges = set(map(tuple, edges))
        # create the set of edges to sample by subtracting the existing
        # edges from the set of possible edges
        sample_edges = all_edges - edges
        # select random edges without replacement based on the noise ratio
        attack_edges = np.random.choice(list(range(len(sample_edges))), 
            size=int(len(edges) * attack_ratio), 
            replace=False)
        attack_edges = [edge for (i, edge) in enumerate(list(sample_edges)) if i in attack_edges]
        # create the new ndarray of edges with the attack edges added
        edges = list(edges) + attack_edges
        return np.array(edges)
    return _attack_edges

In [4]:
X, A, y = load_data('data/cora/', dataset='cora', attack_edges=attack_edges(0))
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

# Model

## Input Layers

In [8]:
X_in = Input(shape=(X.shape[1], ), name='features')
X_in

W1120 15:34:23.692140 139967545513792 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



<tf.Tensor 'features:0' shape=(?, 1433) dtype=float32>

In [9]:
A_in = Input(shape=(None, None), batch_shape=(None, None), sparse=True, name='graph')
A_in

W1120 15:34:23.706912 139967545513792 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



## Training Loop

In [11]:
def train(model, epochs=200, patience=10):
    """
    Train the given model.

    Args:
        model: the model to train
        epochs: the maximum number of training epochs
        patience: the number of patience epochs for early stoping

    Returns:
        a tuple of the testing loss and accuracy

    """
    wait = 0
    preds = None
    best_val_loss = 99999
    # Fit
    progress = tqdm(range(1, epochs + 1))
    for epoch in progress:
        # Single training iteration (we mask nodes without labels for loss calculation)
        model.fit([X, A], y_train, 
            sample_weight=train_mask,
            batch_size=A.shape[0], 
            epochs=1, 
            shuffle=False, 
            verbose=0)
        # Predict on full dataset
        preds = model.predict([X, A], batch_size=A.shape[0])
        # Train / validation scores
        train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val], [idx_train, idx_val])
        # update the progress bar
        progress.set_postfix(train_acc=train_val_acc[0], val_acc=train_val_acc[1])
        # Early stopping
        if train_val_loss[1] < best_val_loss:  # better loss than best
            best_val_loss = train_val_loss[1]
            wait = 0
        elif wait >= patience:  # early stopping
            break
        else:  # waiting for better loss
            wait += 1
    progress.close()
    # return test set evaluation metrics
    return evaluate_preds(preds, [y_test], [idx_test])

## GCN (baseline)

In [12]:
# create the layers
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, 1, 
    activation='relu', 
    kernel_regularizer=l2(5e-4)
)([H, A_in])
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], 1, 
    activation='softmax'
)([H, A_in])
# create the model
gcn = Model(inputs=[X_in, A_in], outputs=Y)
gcn.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['acc'])
gcn.summary()

W1120 15:35:37.766080 139967545513792 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1120 15:35:37.767084 139967545513792 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1120 15:35:37.771519 139967545513792 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1120 15:35:37.780967 139967545513792 deprecation_wrapper.py:119]

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           features[0][0]                   
__________________________________________________________________________________________________
graph (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 graph[0][0]                      
__________

In [13]:
train(gcn)[1]

  0%|          | 0/200 [00:00<?, ?it/s]W1120 15:35:38.061012 139967545513792 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
100%|██████████| 200/200 [00:04<00:00, 42.56it/s, train_acc=0.971, val_acc=0.81] 


[0.803]

## Gaussian Graph Convolution

In [14]:
# create the layers
H = Dropout(0.6)(X_in)
M1, S1 = GaussianGraphConvolution(16,
    is_first=True,
    dropout=0.6
)([H, A_in])
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    last_activation='softmax',
)([M1, S1, A_in])
# create the model
rgcn = Model(inputs=[X_in, A_in], outputs=Y)
rgcn.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
rgcn.add_loss(kl_reg(M1, S1, 5e-4), [M1, S1])
rgcn.summary()

W1120 15:35:42.695752 139967545513792 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1120 15:35:42.718756 139967545513792 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1120 15:35:42.726606 139967545513792 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W1120 15:35:42.752563 139967545513792 deprecation.py:323] From /home/bitcommander/Documents/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/rgcn/ggc.py:206: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.co

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           features[0][0]                   
__________________________________________________________________________________________________
graph (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 graph[0][0]                      
__________

In [15]:
train(rgcn)[1]

 57%|█████▊    | 115/200 [00:02<00:01, 53.97it/s, train_acc=0.95, val_acc=0.793] 


[0.766]